# Data Preparation
## In this section, we will prepare your data for analysis.

#### Import Libraries

In [16]:
import pandas as pd
from glob import glob

#### Concatenation of the csv files
##### Our data was split into 6 different csv files. In this section we are concatenating all these files into one single csv file(base_data.csv).

In [17]:
files = glob("Data_Files/Morticd10_part*.csv")

dfs = []
for f in files:
    df = pd.read_csv(f, low_memory=False)
    dfs.append(df)
merged = pd.concat(dfs, ignore_index=True)

merged.shape

##merged.to_csv("Data_Files/base_data.csv", index=False)

(5143931, 39)

In [5]:
merged['List'].unique()

array(['101', '103', '104', 'UE1', 101, 103, 104, '10M'], dtype=object)

In [11]:
merged["List"].value_counts()
merged["Year"].value_counts()



Year
2016    21340
2014    19957
2013    18514
2015    18196
2017    17899
2008    17638
2005    16706
2012    15654
2011    15603
2004    15284
2007    15187
2019    14797
2009    14779
2006    14744
2010    14711
2000    14177
2018    13647
2001    13258
2003    12549
2002    12461
1998    12449
1999    11416
2020     9338
1997     7930
1996     6930
2022     5437
2021     4981
1995     3011
2023     2966
1994     1978
Name: count, dtype: int64

#### Deletion of unnecessary rows
##### In this section we are deleting all the rows that are not necessary for our analysis.

In [22]:
data = merged[(merged["List"] == 103) | (merged["List"] == '103')]
base_data = data.drop(data.columns[[1, 2]], axis = 1)
base_data.to_csv("Data_Files/base_data.csv", index=False)

#### Adding column headers
##### In this sections, we add a column header in the causes_codes.csv

In [39]:
import pandas as pd

# CSV ohne Header einlesen
df = pd.read_csv("codes_Files/causes_codes.csv", 
                 encoding="utf-8-sig", 
                 sep=";", 
                 header=None)

# Spaltenanzahl ermitteln
n_cols = df.shape[1]

# Leere Header-Liste erzeugen
new_header = [""] * n_cols

# Nur bestimmte Spalten benennen
new_header[6] = "cause_codes"   # 6. Spalte
new_header[9] = "cause_name"    # 8. Spalte

# Header zuweisen
df.columns = new_header

# Speichern mit Header
df.to_csv("codes_Files/causes_codes_with_header.csv", 
          index=False, 
          sep=";", 
          encoding="utf-8-sig")


#### Lookup Country Names
##### In this section we are looking up the country names based on the country codes provided in the dataset (country_codes.csv).

In [42]:
country_codes = pd.read_csv("codes_Files/country_codes.csv", encoding="utf-8-sig")
base_data = pd.read_csv(r"Data_Files/base_data.csv", encoding="utf-8-sig")
causes_codes = pd.read_csv("codes_Files/causes_codes_with_header.csv", encoding="utf-8-sig", sep=";")


#Looking up each country, replacing the code with corresponding name of country
country_lookup = base_data.merge(country_codes, left_on="Country", right_on="country", how="left")
country_lookup["Country"] = country_lookup["name"]
country_lookup = country_lookup.drop(columns=["country", "name"])

country_lookup.to_csv("Data_Files/country_lookup.csv", index=False)
 

#Looking up each disease, replacing the code with corresponding name of disease
causes_lookup = country_lookup.merge(causes_codes, left_on="Cause", right_on="cause_codes", how="left")

causes_lookup["Cause"] = causes_lookup["cause_name"]
 
# Aufräumen
causes_lookup = causes_lookup.drop(columns=["cause_codes", "cause_name"])
 
causes_lookup.to_csv("Data_Files/base_data_with_lookup.csv", index=False)


causes_lookup["Cause"].value_counts()


Cause
Acute myocardial infarction                                                                807
Malignant neoplasm of bronchus and lung                                                    798
Malignant neoplasm of liver and intrahepatic bile ducts                                    795
Other ill-defined and unspecified causes of mortality                                      793
Chronic ischaemic heart disease                                                            792
                                                                                          ... 
Rosacea                                                                                      2
Ophthalmic devices associated with adverse incidents                                         1
Nystagmus and other irregular eye movements                                                  1
Other specified infectious agents as the cause of diseases classified to other chapters      1
Resistance to antineoplastic drugs          

In [15]:
dataset = pd.read_csv("Data_Files/mortality_with_country_names.csv")
dataset = dataset.drop(df.columns[[1, 2]], axis=1)

# save the result
dataset.to_csv("Data_Files/mortality_clean.csv", index=False)

/var/folders/37/084722ds0s5f0lsp5b75z_w40000gn/T/ipykernel_64009/461549012.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("Data_Files/mortality_with_country_names.csv")
